In [1]:
import apache_beam as beam
from apache_beam import CoGroupByKey
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [2]:
p = beam.Pipeline(InteractiveRunner())

In [3]:
m = p | "M" >> beam.Create([('a', 1), ('b', 2)])
ib.collect(m)

,0,1
0,a,1
1,b,2


In [4]:
s = p | "S" >> beam.Create([('b', 10), ('c', 20)])
ib.collect(s)

,0,1
0,b,10
1,c,20


In [5]:
cogbk = (m, s) | CoGroupByKey()
ib.collect(cogbk)

,0,1
0,a,"([1], [])"
1,b,"([2], [10])"
2,c,"([], [20])"


In [6]:
class MySum(beam.DoFn):
    def process(self, element, *args, **kwargs):
        _sum=0
        for _list in element[1]:
            _sum+=sum(_list)
        yield element[0], _sum

In [7]:
join_and_sum = cogbk | beam.ParDo(MySum())

In [8]:
ib.collect(join_and_sum)

,0,1
0,a,1
1,b,12
2,c,20
